### Importação de Bibliotecas
Nesta célula, importamos diversas bibliotecas essenciais para o projeto. Utilizamos `os` para manipulação de diretórios, `librosa` para processamento de áudio, `numpy` e `pandas` para manipulação de dados, `pycaret` para facilitação do processo de modelagem, `sklearn` para o modelo de Regressão Logística e avaliação de desempenho.

In [8]:
import os
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa.display
from pycaret.classification import *
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, mean_squared_error, r2_score)
from sklearn.model_selection import cross_val_score, train_test_split

### Exibição de Espectrograma
Nesta célula, criamos uma função `plot_spectrogram` que exibe o espectrograma de um arquivo de áudio. Usamos `librosa` para carregar o arquivo e calcular o espectrograma de frequência mel, que é exibido usando `matplotlib`. A função é exemplificada com um arquivo da pasta de áudio de ruído.


In [ ]:
def plot_spectrogram(file_path):
    y, sr = librosa.load(file_path, sr=None)
    plt.figure(figsize=(10, 4))
    S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
    S_dB = librosa.power_to_db(S, ref=np.max)
    librosa.display.specshow(S_dB, sr=sr, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel-frequency spectrogram')
    plt.tight_layout()
    plt.show()

### Exibição de Gráfico de Áudio
Nesta célula, criamos uma função `plot_waveform` que exibe o gráfico de um arquivo de áudio. Usamos `librosa` para carregar o arquivo e `matplotlib` para exibir a forma de onda. A função é exemplificada com o mesmo arquivo da célula anterior.


In [ ]:
def plot_waveform(file_path):
    y, sr = librosa.load(file_path, sr=None)
    plt.figure(figsize=(10, 4))
    librosa.display.waveshow(y, sr=sr)
    plt.title('Waveform')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.tight_layout()
    plt.show()

### Definição dos Caminhos e Função para Extração de Features
Definimos os caminhos para os diretórios contendo os áudios das classes 'ruído' e 'barco'. A função `extract_features` é responsável por extrair várias características dos arquivos de áudio usando a biblioteca `librosa`. Esses recursos incluem medidas espectrais e MFCCs (Mel-Frequency Cepstral Coefficients), que são comumente usados na análise de áudio. Em seguida, combinamos os dados extraídos e os salvamos em um arquivo CSV.

In [9]:
# Caminhos para as pastas de áudio
pasta_ruido = r'../audios/audios_treinamento/classe_ruido'
pasta_barco = r'../audios/audios_treinamento/classe_barco'

# Função para extrair features dos áudios
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    features = {
        'chroma_stft': librosa.feature.chroma_stft(y=y, sr=sr).mean(),
        'chroma_cqt': librosa.feature.chroma_cqt(y=y, sr=sr).mean(),
        'chroma_cens': librosa.feature.chroma_cens(y=y, sr=sr).mean(),
        'rmse': librosa.feature.rms(y=y).mean(),
        'spectral_centroid': librosa.feature.spectral_centroid(y=y, sr=sr).mean(),
        'spectral_bandwidth': librosa.feature.spectral_bandwidth(y=y, sr=sr).mean(),
        'spectral_contrast': librosa.feature.spectral_contrast(y=y, sr=sr).mean(),
        'spectral_flatness': librosa.feature.spectral_flatness(y=y).mean(),
        'spectral_rolloff': librosa.feature.spectral_rolloff(y=y, sr=sr).mean(),
        'zero_crossing_rate': librosa.feature.zero_crossing_rate(y).mean(),
        'mfcc1': librosa.feature.mfcc(y=y, sr=sr)[0].mean(),
        'mfcc2': librosa.feature.mfcc(y=y, sr=sr)[1].mean(),
        'mfcc3': librosa.feature.mfcc(y=y, sr=sr)[2].mean(),
        # 'mfcc4': librosa.feature.mfcc(y=y, sr=sr)[3].mean(),
        # 'mfcc5': librosa.feature.mfcc(y=y, sr=sr)[4].mean(),
        # 'mfcc6': librosa.feature.mfcc(y=y, sr=sr)[5].mean(),
        # 'mfcc7': librosa.feature.mfcc(y=y, sr=sr)[6].mean(),
        # 'mfcc8': librosa.feature.mfcc(y=y, sr=sr)[7].mean(),
        # 'mfcc9': librosa.feature.mfcc(y=y, sr=sr)[8].mean(),
        # 'mfcc10': librosa.feature.mfcc(y=y, sr=sr)[9].mean(),
        # 'mfcc11': librosa.feature.mfcc(y=y, sr=sr)[10].mean(),
        # 'mfcc12': librosa.feature.mfcc(y=y, sr=sr)[11].mean(),
        # 'mfcc13': librosa.feature.mfcc(y=y, sr=sr)[12].mean(),
        #'tempo': librosa.beat.tempo(y=y, sr=sr)[0]
    }
    return features

# Função para processar as pastas e adicionar os dados ao DataFrame
def verifica_pastas(folder_path, label):
    data = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_path.endswith('.wav'):
            features = extract_features(file_path)
            features['label'] = label
            data.append(features)
    return data

# Processar as duas pastas sendo cada uma uma classe
dados_ruido = verifica_pastas(pasta_ruido, 1)
dados_barco = verifica_pastas(pasta_barco, 2)

# Combinar os dados e criar um DataFrame
data = dados_ruido + dados_barco
df = pd.DataFrame(data)

# Salvar em um arquivo CSV
df.to_csv("csv/features_audios_classificados.csv", index=False)

### Exibição do Espectrograma

In [ ]:
file_path = os.path.join(pasta_ruido, os.listdir(pasta_ruido)[0])
plot_spectrogram(file_path)

### Exibição de Gráfico de Áudio

In [ ]:
plot_waveform(file_path)

### Treinamento do Modelo de Regressão Logística
Carregamos os dados salvos no CSV e separamos os atributos (`X`) e os rótulos (`y`). Criamos uma instância do modelo `LogisticRegression` e a treinamos com os dados. Em seguida, avaliamos o modelo utilizando validação cruzada, calculando a precisão média das predições.

In [10]:
dados = pd.read_csv("csv/features_audios_classificados.csv")
X = dados.drop(columns=['label'])
y = dados['label']

# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Treinando o modelo LDA
lr_model = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
lr_model.fit(X_train, y_train)

scores = cross_val_score(lr_model, X, y, cv=5)
print(f"Cross-validation scores: {scores}")
print(f"Mean cross-validation score: {np.mean(scores)}")

Cross-validation scores: [0.85714286 0.92857143 1.         1.         1.        ]
Mean cross-validation score: 0.9571428571428571


### Avaliação Adicional do Modelo
Dividimos o conjunto de dados em treinamento e teste para realizar uma avaliação adicional do modelo. Treinamos o modelo com o conjunto de treinamento e geramos previsões para o conjunto de teste. O relatório de classificação é gerado para avaliar o desempenho do modelo em termos de métricas como precisão, recall e F1-score.


In [11]:
# Fazer previsões e avaliar o modelo
y_pred = lr_model.predict(X_test)
print("Relatório de Classificação para LDA:")
print(classification_report(y_test, y_pred))

Relatório de Classificação para LDA:
              precision    recall  f1-score   support

           1       0.93      1.00      0.97        14
           2       1.00      0.95      0.97        20

    accuracy                           0.97        34
   macro avg       0.97      0.97      0.97        34
weighted avg       0.97      0.97      0.97        34



### Configuração do Ambiente e Salvamento do Modelo
Utilizamos o `setup` do `pycaret` para configurar o ambiente de classificação com os dados e o alvo definidos. Finalmente, salvamos o modelo treinado usando a função `save_model` do `pycaret`.

In [12]:
clf1 = setup(data=dados, target='label', session_id=123)
save_model(lr_model, 'Modelo_LR')

,Description,Value
0,Session id,123
1,Target,label
2,Target type,Binary
3,Target mapping,"1: 0, 2: 1"
4,Original data shape,"(67, 14)"
5,Transformed data shape,"(67, 14)"
6,Transformed train set shape,"(46, 14)"
7,Transformed test set shape,"(21, 14)"
8,Numeric features,13
9,Preprocess,True


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['chroma_stft', 'chroma_cqt',
                                              'chroma_cens', 'rmse',
                                              'spectral_centroid',
                                              'spectral_bandwidth',
                                              'spectral_contrast',
                                              'spectral_flatness',
                                              'spectral_r...
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                    